In [1]:
import yaml
import os
import numpy as np
import copy

In [2]:
base_config={
                   #'model_name': ,
                   #'model_path': ,
                   #'device': ,
                   'class_groups': [[2*i,2*i+1] for i in range(5)],
                   #'dataset_name':'MNIST' or 'CIFAR' ,
                   #'dataset_type': 'std', 'group', 'corrupt' or 'mark',
                   #'data_root':,
                   #'xpl_root':,
                   #'coef_root':,
                   #'save_dir': ,
                   'validation_size': 2000,
}
project_root="C:/Users/weckbecker/DualView-wip"
dataset_root="C:/Users/weckbecker/DualView-wip/src/datasets"
data_root="C:/Users/weckbecker/DualView-wip/src/datasets"

In [3]:
def create_config(config, config_name):
    if config['dataset_type'] == 'stdk':
        dsname_file = 'std'
    elif config['dataset_type'] == 'groupk':
        dsname_file = 'group'
    elif config['dataset_type'] == 'switched':
        dsname_file = 'switched_one_file'
    elif config['dataset_type'] == 'add_batch_in':
        dsname_file = 'std'
    else:
        dsname_file = config['dataset_type']
    config['data_root'] = data_root
    config['xpl_root'] = f"{project_root}/explanations/{config['dataset_name']}/{dsname_file}/{config['model_name']}_{dsname_file}/{config['xai_method']}"
    if config['xai_method']=='dualview':
        config['C']=0.001
        config['xpl_root']=f"{config['xpl_root']}_{config['C']}"
    if config['xai_method'] in ['dualview', 'representer']:
        config['coef_root']=config['xpl_root']

    config['device'] = 'cuda'
    if config['dataset_type']=="switched":
        config['model_path']=f"{project_root}/checkpoints/{config['dataset_name']}/std/{config['model_name']}_std/{config['dataset_name']}_{config['model_name']}"
    else:
        config['model_path']=f"{project_root}/checkpoints/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['dataset_name']}_{config['model_name']}"
    config['save_dir'] = f'{project_root}/test_output'
    path = f"evaluate/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    
    with open(f"{path}/{config_name}.yaml", 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)

In [4]:
names=[('MNIST', 'basic_conv'),('CIFAR', 'resnet'), ('AWA', 'resnet')]
dataset_types=['std', 'group', 'corrupt', 'mark',
               'stdk', 'groupk', 'switched', 'add_batch_in',
               "lds", "labelflip"]
xai_methods=['dualview', 'representer', 'influence', 'rp_similarity', 'similarity']

In [5]:
from copy import deepcopy
for dsname, model_name in names:
    base_config['model_name']=model_name   
    base_config['dataset_name']=dsname
    for dstype in dataset_types:
        if (dstype=='group') or (dstype=='groupk'):
            base_config['num_classes']=len(base_config['class_groups'])
        else:
            base_config['num_classes']=10
        base_config['dataset_type']=dstype
        for xai_method in xai_methods:
            base_config['xai_method']=xai_method
            create_config(deepcopy(base_config), f"{model_name}_{dstype}_{xai_method}")